In [ ]:
# IMPORTING LIBRARIES, PACKAGES, SETTING WORKING DIRECTORIES AND DEFINING VARIABLES

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import pafy
import os
import np_utils
from sklearn.cross_validation import train_test_split
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, Adam, RMSprop

Main_Folder='/Users/gautamroy/Documents/My Python Package/Python OpenCV'
Training_Folder='/Users/gautamroy/Documents/My Python Package/Python OpenCV/Training_set'
Validation_Folder='/Users/gautamroy/Documents/My Python Package/Python OpenCV/Validation_set'

OPTIMIZER = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 5
N_HIDDEN = 128
SPLIT_RATIO = 0.2
IMG_CHANNELS = 3
IMG_ROWS = 540
IMG_COLS = 960
DROPOUT = 0.3


In [ ]:
# CREATING ARRAY FOR THE TRANING AND VALIDATION SET FROM IMAGE FRAMES

os.chdir(Main_Folder)

Train_Frame_set=pd.read_csv('Train_Frame_set.csv')
Validation_Frame_set=pd.read_csv('Validation_Frame_set.csv')

os.chdir(Training_Folder)

X_train=np.array([cv2.imread(i, 3) for i in Train_Frame_set.loc[:,'FrameID']])
y_train=np.array([i for i in Train_Frame_set.loc[:,'Category']])

os.chdir(Validation_Folder)

X_valid=np.array([cv2.imread(i, 3) for i in Validation_Frame_set.loc[:,'FrameID']])
y_valid=np.array([i for i in Validation_Frame_set.loc[:,'Category']])


In [ ]:
# Resize Arrays: Set width, height same for all images

X_train=np.array([cv2.resize(img,(IMG_COLS, IMG_ROWS), interpolation = cv2.INTER_LINEAR) for img in X_train])
X_valid=np.array([cv2.resize(img,(IMG_COLS, IMG_ROWS), interpolation = cv2.INTER_LINEAR) for img in X_valid])

# Blurring the images to reduce noise

X_train=np.array([cv2.GaussianBlur(img,(5,5),0) for img in X_train])
X_valid=np.array([cv2.GaussianBlur(img,(5,5),0) for img in X_valid])

In [ ]:
# MODEL BUILDING USING DATA AUGUMENTATION

train_datagen = ImageDataGenerator(
        rescale=1./255,
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(
        rescale=1./255)

train_datagen.fit(X_train)
valid_datagen.fit(X_valid)

In [ ]:
# DESIGNING THE KERAS MODEL

def MODEL_DEFINATION():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS),data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),data_format='channels_last'))
    model.add(Dropout(DROPOUT))

    model.add(Conv2D(64, (3, 3), padding='same',data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),data_format='channels_last'))
    model.add(Dropout(DROPOUT))

    model.add(Conv2D(64, (3, 3), padding='same',data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),data_format='channels_last'))
    model.add(Dropout(DROPOUT))


    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(DROPOUT))
    model.add(Dense(NB_CLASSES))
    model.add(Activation(tf.nn.softmax))

    return model


In [ ]:
# EXECUTING THE KERAS MODEL USING DATA AUGUMENTATION

model=MODEL_DEFINATION()

model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])

Trained_model = model.fit_generator(
        train_datagen.flow(X_train, y_train, batch_size=32),
        steps_per_epoch=int(np.ceil(X_train.shape[0] / float(BATCH_SIZE))), 
        epochs=NB_EPOCH, 
        verbose=VERBOSE, 
        validation_data=(X_valid, y_valid))
